In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
import inspect
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python


### Get recording info (google sheets)

In [3]:
import requests
import pandas as pd
from io import BytesIO

In [4]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [5]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10


### Submit job

In [6]:
from multicamera_airflow_pipeline.tim_240731.interface.o2 import O2Runner
from pathlib import Path
from datetime import datetime

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/n/groups/datta/tim_sainburg/conda_envs/peromoseq/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,


In [7]:
for idx, recording_row in recording_df.iterrows():
    break

In [8]:
recording_row

Subject                                                                  M04002
duration_m                                                                   10
video_recording_id                                     24-05-01-13-26-43-110846
ephys_id                                                    2024-05-01_13-26-37
calibration_id                                         24-05-01-13-45-07-825493
video_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
ephys_location_on_o2          /n/groups/datta/tim_sainburg/datasets/chronic2...
calibration_location_on_o2    /n/groups/datta/tim_sainburg/datasets/chronic2...
samplerate                                                                  150
username                                                                 tis697
n_ephys_streams                                                               1
max_video_duration_m                                                         10
Name: 0, dtype: object

### Job specific

In [9]:
import yaml
import textwrap
import time

In [10]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/240806-3d-pipeline

2D_predictions	    continuous_features   size_normalization  triangulation
arena_alignment     egocentric_alignment  spikesorting	      triangulation2
camera_calibration  gimbal		  tensorrt
camera_sync	    openephys_sync	  tensorrt-2


In [11]:
# these are arguments
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"
job_directory = Path('/n/groups/datta/tim_sainburg/datasets/scratch/jobs')
job_directory.mkdir(exist_ok=True, parents=True)
config_file = Path("/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml")

In [12]:
def convert_minutes_to_hms(minutes_float):
    # Convert minutes to total seconds
    total_seconds = int(minutes_float * 60)
    
    # Extract hours, minutes, and seconds using divmod
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    
    # Format as HH:MM:SS
    return f"{hours:02}:{minutes:02}:{seconds:02}"

In [13]:
def check_spikesorting_completion(spikesorting_output_directory, n_ephys_streams_expected):
    n_sorts_found = len(list(spikesorting_output_directory.glob('*/sort_result')))
    if n_sorts_found == n_ephys_streams_expected:
        return True
    else:
        return False

In [14]:
n_ephys_streams_expected = recording_row.n_ephys_streams

In [15]:
# load config
config_file = Path(config_file)
config = yaml.safe_load(open(config_file, "r"))

In [16]:
# where the video data is located
recording_directory = Path(recording_row.calibration_location_on_o2) / recording_row.calibration_id

In [17]:
# where to save output
spikesorting_output_directory = (
    output_directory / "spikesorting" / recording_row.video_recording_id
)
spikesorting_output_directory.mkdir(parents=True, exist_ok=True)
current_datetime_str = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
remote_job_directory = job_directory / current_datetime_str

In [18]:
# check if sync successfully completed
if config["spikesorting"]["recompute_completed"] == False:
    if (
        check_spikesorting_completion(spikesorting_output_directory, n_ephys_streams_expected)
    ):
        print("spikesorting completed, quitting")
        # return
    else:
        print("spikesorting incomplete, starting")

spikesorting incomplete, starting


In [19]:
duration_requested = convert_minutes_to_hms(
        recording_row.duration_m * config["o2"]["spikesorting"]["o2_runtime_multiplier"]
    )
duration_requested

'00:40:00'

In [20]:
samplerate = recording_row.samplerate

In [21]:
ephys_recording_directory = Path(recording_row.ephys_location_on_o2) / recording_row.ephys_id

In [22]:
params = {
    "spikesorting_output_directory": spikesorting_output_directory.as_posix(),
    "ephys_recording_directory": ephys_recording_directory.as_posix()
}

In [23]:
# create the job runner
runner = O2Runner(
    job_name_prefix=f"{recording_row.video_recording_id}_spikesorting",
    remote_job_directory=remote_job_directory,
    conda_env="/n/groups/datta/tim_sainburg/conda_envs/kilosort4",
    o2_username=recording_row.username,
    o2_server="login.o2.rc.hms.harvard.edu",
    job_params=params,
    o2_n_cpus=config["o2"]["spikesorting"]["o2_n_cpus"],
    o2_memory=config["o2"]["spikesorting"]["o2_memory"],
    o2_time_limit=duration_requested,
    o2_queue=config["o2"]["spikesorting"]["o2_queue"],
    o2_exclude=config["o2"]["spikesorting"]["o2_exclude"],
    o2_qos = config["o2"]["spikesorting"]["o2_qos"] ,
    o2_gres = config["o2"]["spikesorting"]["o2_gres"] ,
)

INFO:paramiko.transport:Connected (version 2.0, client OpenSSH_7.4)
INFO:paramiko.transport:Auth banner: b'Problems logging in?\nUse your lower case HMS ID, like abc123, not ABC123.\nIf locked out, see:\nhttps://it.hms.harvard.edu/i-want/reset-password-or-unlock-your-hms-account\n'
INFO:paramiko.transport:Authentication (publickey) successful!


In [24]:
runner.python_script = textwrap.dedent(
f"""
# load params
import yaml
params_file = "{runner.remote_job_directory / f"{runner.job_name}.params.yaml"}"
config_file = "{config_file.as_posix()}"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

# grab sync cameras function
from multicamera_airflow_pipeline.tim_240731.ephys.spikesorting_ks4 import SpikeSorter
spikesorter = SpikeSorter(
    ephys_recording_directory= params["ephys_recording_directory"],
    spikesorting_output_directory = params["spikesorting_output_directory"],
    **config["spikesorting"]
)
spikesorter.run()


"""
)

In [25]:
print(runner.python_script)


# load params
import yaml
params_file = "/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_155629_670825/24-05-01-13-26-43-110846_spikesorting_24-08-09-2024-56-29-743272.params.yaml"
config_file = "/n/groups/datta/tim_sainburg/projects/multicamera_airflow_pipeline/multicamera_airflow_pipeline/tim_240731/default_config.yaml"

params = yaml.safe_load(open(params_file, 'r'))
config = yaml.safe_load(open(config_file, 'r'))

# grab sync cameras function
from multicamera_airflow_pipeline.tim_240731.ephys.spikesorting_ks4 import SpikeSorter
spikesorter = SpikeSorter(
    ephys_recording_directory= params["ephys_recording_directory"],
    spikesorting_output_directory = params["spikesorting_output_directory"],
    **config["spikesorting"]
)
spikesorter.run()





In [26]:
runner.run()

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote job directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_155629_670825
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Creating remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_155629_670825
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Successfully created remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_155629_670825
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Writing job files to remote directory: /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_155629_670825
INFO:paramiko.transport.sftp:[chan 1] Opened sftp connection (server version 3)
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Transferring /tmp/tmpnc4bo35e to login.o2.rc.hms.harvard.edu:/n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_155629_670825/24-05-01-13-26-43-110846_spikesorting_24-08-09-2024-56-29-74327

In [27]:
# wait until the job is finished
# 10000/60/24 = roughly 1 week
for i in range(10000):
    # check job status every n seconds
    status = runner.check_job_status()
    if status:
        break
    time.sleep(60)

INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43757116
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Unknown job state: 
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43757116
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43757116
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43757116
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43757116
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:The job is waiting to be scheduled.
INFO:multicamera_airflow_pipeline.tim_240731.interface.o2:Checking job status: 43757116
INFO:multicamera_a

In [28]:
!scancel 43757116

/bin/bash: line 1: scancel: command not found


In [ ]:
# check if sync successfully completed
if check_spikesorting_completion(spikesorting_output_directory, n_ephys_streams_expected):
    print('spikesorting completed successfully')
else:
    raise ValueError("spikesorting did not complete successfully.")


In [ ]:
!ls /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_142026_932371/

In [ ]:
!tail /n/groups/datta/tim_sainburg/datasets/scratch/jobs/20240809_142026_932371/24-05-01-13-26-43-110846_spikesorting_24-08-09-2024-20-28-025140.log